In [38]:
import sys
import MDAnalysis as mda
import prolif as plf
import prolif
from rdkit import Chem
from rdkit import Geometry
import py3Dmol
from prolif.plotting.network import LigNetwork
import numpy as np


'''
conda create -n prolif
conda activate prolif
conda config --add channels conda-forge
conda install rdkit cython
pip install git+https://github.com/chemosim-lab/ProLIF.git
pip install py3Dmol
'''


def get_ring_centroid(mol, index):
    # find ring using the atom index
    Chem.SanitizeMol(mol, Chem.SanitizeFlags.SANITIZE_SETAROMATICITY)
    ri = mol.GetRingInfo()
    for r in ri.AtomRings():
        if index in r:
            break
    else:
        raise ValueError("No ring containing this atom index was found in the given molecule")
    # get centroid
    coords = mol.xyz[list(r)]
    ctd = plf.utils.get_centroid(coords)
    return Geometry.Point3D(*ctd)


def GenerateHostGuestInteractionFingerprint(top,traj,firstframe,lastframe,framestep):
    u = mda.Universe(top,traj)
    prot = u.select_atoms("protein")
    lig = u.select_atoms("resname LIG")
    fp = prolif.Fingerprint(['Hydrophobic','HBDonor', 'HBAcceptor', 'PiStacking', 'Anionic', 'Cationic', 'CationPi', 'PiCation'])
    fp.run(u.trajectory[firstframe:lastframe:framestep], lig, prot)
    df = fp.to_dataframe(return_atoms=True)
    print(df)
    lmol = plf.Molecule.from_mda(lig)
    pmol = plf.Molecule.from_mda(prot)
    return lmol,pmol,df,u




In [39]:
traj='C:/Users/brand/OneDrive/Desktop/melk_CompE1_V1_R0_compboxproddyn_trim.arc'
top='C:/Users/brand/OneDrive/Desktop/topologyguess.pdb'
framestep=100
firstframe=50
lastframe=51
lmol,pmol,df,u=GenerateHostGuestInteractionFingerprint(top,traj,firstframe,lastframe,framestep)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.99s/it]


ligand             LIG2                                                  \
protein         CYS71.B    GLU118.B     GLU75.B      GLY2.B     GLY74.B   
interaction Hydrophobic Hydrophobic Hydrophobic Hydrophobic Hydrophobic   
Frame                                                                     
50               (7, 6)     (34, 2)     (34, 6)      (9, 1)     (13, 1)   

ligand                                                                   
protein        ILE131.B    LEU121.B     PRO72.B     TYR70.B      VAL7.B  
interaction Hydrophobic Hydrophobic Hydrophobic Hydrophobic Hydrophobic  
Frame                                                                    
50              (32, 9)      (4, 9)     (18, 2)      (7, 8)      (7, 7)  


In [40]:
colors = {
    "Hydrophobic": "red",
    "HBDonor": "orange",
    "HBAcceptor": "yellow",
    "PiStacking": "green",
    "Anionic": "blue",
    "Cationic": "indigo",
    "CationPi": "purple",
    "PiCation": "black",
}

for interaction,color in colors.items():
    print('Interaction:'+interaction+' Color:'+color)
    
# JavaScript functions
resid_hover = """function(atom,viewer) {{
    if(!atom.label) {{
        atom.label = viewer.addLabel('{0}:'+atom.atom+atom.serial,
            {{position: atom, backgroundColor: 'mintcream', fontColor:'black'}});
    }}
}}"""
hover_func = """
function(atom,viewer) {
    if(!atom.label) {
        atom.label = viewer.addLabel(atom.interaction,
            {position: atom, backgroundColor: 'black', fontColor:'white'});
    }
}"""
unhover_func = """
function(atom,viewer) {
    if(atom.label) {
        viewer.removeLabel(atom.label);
        delete atom.label;
    }
}"""

v = py3Dmol.view(650, 600)
v.removeAllModels()

models = {}
mid = -1
for i, row in df.T.iterrows():
    lresid, presid, interaction = i
    lindex, pindex = row[firstframe]
    lres = lmol[lresid]
    pres = pmol[presid]
    # set model ids for reusing later
    for resid, res, style in [(lresid, lres, {"colorscheme": "cyanCarbon"}),
                              (presid, pres, {})]:
        if resid not in models.keys():
            mid += 1
            v.addModel(Chem.MolToMolBlock(res), "sdf")
            model = v.getModel()
            model.setStyle({}, {"stick": style})
            # add residue label
            model.setHoverable({}, True, resid_hover.format(resid), unhover_func)
            models[resid] = mid
    # get coordinates for both points of the interaction
    if interaction in ["PiStacking", "EdgeToFace", "FaceToFace", "PiCation"]:
        p1 = get_ring_centroid(lres, lindex)
    else:
        p1 = lres.GetConformer().GetAtomPosition(lindex)
    if interaction in ["PiStacking", "EdgeToFace", "FaceToFace", "CationPi"]:
        p2 = get_ring_centroid(pres, pindex)
    else:
        p2 = pres.GetConformer().GetAtomPosition(pindex)
    # add interaction line
    v.addCylinder({"start": dict(x=p1.x, y=p1.y, z=p1.z),
                   "end":   dict(x=p2.x, y=p2.y, z=p2.z),
                   "color": colors[interaction],
                   "radius": .15,
                   "dashed": True,
                   "fromCap": 1,
                   "toCap": 1,
                  })
    # add label when hovering the middle of the dashed line by adding a dummy atom
    c = Geometry.Point3D(*plf.utils.get_centroid([p1, p2]))
    modelID = models[lresid]
    model = v.getModel(modelID)
    model.addAtoms([{"elem": 'Z',
                     "x": c.x, "y": c.y, "z": c.z,
                     "interaction": interaction}])
    model.setStyle({"interaction": interaction}, {"clicksphere": {"radius": .5}})
    model.setHoverable(
        {"interaction": interaction}, True,
        hover_func, unhover_func)

# show protein:
# first we need to reorder atoms as in the original MDAnalysis file.
# needed because the RDKitConverter reorders them when infering bond order
# and 3Dmol.js doesn't like when atoms from the same residue are spread accross the whole file
order = np.argsort([atom.GetIntProp("_MDAnalysis_index") for atom in pmol.GetAtoms()])
mol = Chem.RenumberAtoms(pmol, order.astype(int).tolist())
mol = Chem.RemoveAllHs(mol)
pdb = Chem.MolToPDBBlock(mol, flavor=0x20 | 0x10)
v.addModel(pdb, "pdb")
model = v.getModel()
model.setStyle({}, {"cartoon": {"style":"edged"}})

for res in u.atoms[:].residues:
    resid=res.resid
    resname=res.resname
    #if resname!='UNL':
        #v.addLabel(resname,{'fontOpacity':1},{'resi':int(resid)})
    
v.zoomTo({"model": list(models.values())})

Interaction:Hydrophobic Color:red
Interaction:HBDonor Color:orange
Interaction:HBAcceptor Color:yellow
Interaction:PiStacking Color:green
Interaction:Anionic Color:blue
Interaction:Cationic Color:indigo
Interaction:CationPi Color:purple
Interaction:PiCation Color:black


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [34]:
net = LigNetwork.from_ifp(df, lmol,
                              # replace with `kind="frame", frame=0` for the other depiction
                              kind="aggregate", threshold=.3,
                              rotation=270)
net.display()


C:\Users\brand\miniconda3\envs\prolif\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
